In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.linalg import toeplitz
from ipywidgets import interact
from tqdm import tqdm

# Reaction-Diffusion systems

\begin{align*}
\partial_t u &= D_u \nabla^2 u + R_u(u,v)\\
\partial_t v &= D_v \nabla^2 v + R_v(u,v)
\end{align*}

In [ ]:
# Brusselator
Du, Dv = 4, 10
A ,B= 3,9
def reactions(u, v):
    return (A - (B+1)*u + u**2*v, B*u - u**2*v)
#     return (0,0)

In [ ]:
# pattern scale
L = 10*np.pi
ell = 2*np.pi / np.sqrt((B-1)/(2*Du) - A**2/(2*Dv))
print("Pattern scale =", L/ell)

In [ ]:
# RHS of equation
def derivative(t, y, D2):
    u1, u2 = np.split(y, 2)
    R1, R2 = reactions(u1, u2)
    du1dt = Du*np.dot(D2, u1) + R1
    du2dt = Dv*np.dot(D2, u2) + R2
    return np.concatenate([du1dt, du2dt])

In [ ]:
tmax = 100
Nx, Nt = 100, 500

# space-time grid points
x = np.linspace(0, L, Nx, endpoint=False)
times = np.linspace(0, tmax, Nt)

In [ ]:
# second derivative Toeplitz matrix
dx = x[1] - x[0]
z = np.zeros(x.size)
z[0], z[1], z[-1] = -2, 1, 1
D2 = toeplitz(z) / dx**2

In [ ]:
# vector of function values
u = np.zeros(Nx)
v = np.zeros(Nx)

# initial conditions
u0 = 3*np.ones(x.size) + 0.01 * np.random.randn(x.size)
v0 = 3*np.ones(x.size) + 0.01 * np.random.randn(x.size)
# u0 = 3*np.ones(x.size)
# v0 = 3*np.ones(x.size)

y0 = np.concatenate([u0, v0])

In [ ]:
# solve the diffusion equation
sol = solve_ivp(derivative, t_span=[0, tmax], y0=y0, t_eval=times, args=(D2,), method='RK45')
u, v = np.split(sol.y, 2, axis=0)

In [ ]:
cmin, cmax = sol.y.min(), sol.y.max()
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8,6), dpi=75)
cf1 = ax1.contourf(x, times, u.transpose(), vmin=cmin, vmax=cmax)
fig.colorbar(cf1)
fig.align_xlabels()
cf2 = ax2.contourf(x, times, v.transpose(), vmin=cmin, vmax=cmax)
fig.colorbar(cf2)
plt.savefig(fname='Brusselator')

In [ ]:
# for i in tqdm(range(168,172)):
for i in tqdm(range(times.size)):
    plt.plot(x,u[:,i])
    plt.plot(x,v[:,i])
#     plt.ylim(0,5)
    plt.xlabel("Position(x)")
    plt.ylabel("Concentration")
    plt.legend(["u","v"], loc=1)
    plt.savefig(fname=str(int(i))+'.png')
    plt.pause(0.01)
    plt.clf()